# LESSON 2: Informed Search Strategies

In the second session we will work on informed search

### Maze Environments
The environments used is **SmallMaze** (visible in the figure).

<img src="images/maze.png" width="300">

The agent starts in cell $(0, 2)$ and has to reach the treasure in $(4, 3)$.

### Priority Queue

You will need a queue ordered by priority as queue, or **PriorityQueue**. The difference between the other versions of queue is that in **PriorityQueue** nodes are removed from the queue based on the current lowest value. In particular, **Node** has two useful parameters (other than those used in the previous session):

- pathcost - the path cost from the root node to the current one (defaults to 0)
- value - value of a node. Used by PriorityQueue to order its content (defaults to 0)

Here is an example of usage:

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../tools'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.ai_lab_functions import *

# Create 3 nodes for state ids 1 2 3
node_1 = Node(1) # No parent, pathcost=0, value=0
node_2 = Node(2, node_1, node_1.pathcost + 1, 10) # Child of node_1, pathcost=1, value=10
node_3 = Node(3, node_1, 100, 5)  # Child of node_1, pathcost=100, value=5

p_queue = PriorityQueue()
for n in (node_1, node_2, node_3):
    p_queue.add(n)
    print("Added: {}".format(n.state))

while not p_queue.is_empty():
    print("Removed: {}".format(p_queue.remove().state))

Added: 1
Added: 2
Added: 3
Removed: 1
Removed: 3
Removed: 2


Notice the order in which nodes are removed from the queue.

### Uniform-Cost Search (UCS)
Before moving to informed search it is useful to see another uninformed search algorithm: the Uniform-Cost Search (UCS). In the following you can see the implementation in tree search version. Cost of performing an action is supposd to be 1 (also in the assignements).

In [2]:
import gym
import envs
from utils.ai_lab_functions import build_path


def ucs(environment):
    """
    Uniform-cost search
    
    Args:
        environment: OpenAI Gym environment
        
    Returns:
        path: solution as a path
    """
    queue = PriorityQueue()
    queue.add(Node(environment.startstate))
    while True:
        if queue.is_empty():
            return None
        node = queue.remove()  # Retrieve node from the queue
        if node.state == environment.goalstate:  # Goal state check
            return build_path(node)
        for action in range(environment.action_space.n):  # Look around
            # Child node where value and pathcost are both the pathcost of parent + 1
            child = Node(environment.sample(node.state, action), node, node.pathcost + 1, node.pathcost + 1)
            queue.add(child)

# Create and render the environment
env = gym.make("SmallMaze-v0")
env.render()
solution = ucs(env)

# Print path
print("\nSolution: {}".format([env.state_to_pos(s) for s in solution]))

[['C' 'C' 'S' 'C']
 ['C' 'C' 'W' 'C']
 ['C' 'C' 'C' 'C']
 ['C' 'W' 'W' 'W']
 ['C' 'C' 'C' 'G']]

Solution: [(0, 1), (1, 1), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]


### Distance Heuristics

Informed search requires a distance heuristic to be used in order to estimate the distance between a state and the goal. You already have at your disposal these functions:

- *l1_norm(p1, p2)* - Computes the L1 norm (also known as the manhattan distance) between two points specified as tuples of coordinates
- *l2_norm(p1, p2)* - Computes the L2 norm between two points specified as tuples of coordinates
- *chebyshev(p1, p2)* - Computes the Chebyshev distance between two points specified as tuples of coordinates. Similar to L1 norm but diagonal moves are also considered


Examples:

In [3]:
p1 = (0, 2)
p2 = (4, 0)
print("L1 norm heuristic value: {}".format(Heu.l1_norm(p1, p2)))
print("L2 norm heuristic value: {}".format(Heu.l2_norm(p1, p2)))
print("Chebyshev heuristic value: {}".format(Heu.chebyshev(p1, p2)))

L1 norm heuristic value: 6
L2 norm heuristic value: 4.47213595499958
Chebyshev heuristic value: 4


## Assignment 1: Greedy Best-First Search

The first assignment is to implement the Greedy-best-first search algorithm on **SmallMaze**. In particular, you are required to implement both *greedy_tree_search* and *greedy_graph_search* versions that will be called by the generic *greedy*. Use L1 norm as heuristic function at first, then try also the others to see the differences.

The results returned by greedy must be in the following form (path, time_cost, space_cost), more in detail:

- **path** - tuple of state identifiers forming a path from the start state to the goal state. None if no solution is found.
- **time_cost** - the number of nodes checked during the exploration.
- **space_cost** - the maximum number of nodes in memory at the same time.

Functions to implement:
- *greedy_tree_search(environment)*
- *greedy_graph_search(environment)*

In [20]:
def greedy_tree_search(environment, timeout=10000):
    """
    Greedy-best-first Tree search

    Args:
        problem: OpenAI Gym environment

    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """

    goalpos = environment.state_to_pos(environment.goalstate)
    start_pos = environment.startstate

    # Mahnattan distance.
    heu = Heu.l1_norm(environment.state_to_pos(start_pos), goalpos)

    # The node with STATE = problem.INITIAL-STATE.
    first_node = Node(environment.startstate, None, 0, heu)
    
    # Create and initialize the frontier.
    frontier = PriorityQueue()
    frontier.add(first_node)

    time_cost = 0
    space_cost = 1

    while True:
        # Timeout check.
        if time_cost >= timeout:
            return ("time-out", time_cost, space_cost)

        # Check if EMPTY?(frontier).
        if frontier.is_empty():
            return None, time_cost, space_cost

        # node <- POP(frontier).
        node = frontier.remove()
        time_cost += 1
        space_cost -= 1

        # Check if node.STATE is a problem.GOAL.
        if node.state == environment.goalstate:
            return build_path(node), time_cost, space_cost

        # For each action in problem.ACTIONS(environment.ACTION-SPACE) (look around).
        for action in range(environment.action_space.n):
            next_move = environment.sample(node.state, action)
            pos = environment.state_to_pos(next_move)
            Heu.l1_norm(pos, goalpos)

            # Path cost and euristic as node value.
            child = Node(next_move, node, node.pathcost + 1, heu)

            # frontier <- INSERT(child, frontier).
            frontier.add(child)
            space_cost += 1

In [23]:
def greedy_graph_search(environment):
    """
    Greedy-best-first Graph search

    Args:
        problem: OpenAI Gym environment

    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """

    goalpos = environment.state_to_pos(environment.goalstate)
    start_pos = environment.startstate

    # Mahnattan distance.
    heu = Heu.l1_norm(environment.state_to_pos(start_pos), goalpos)

    # The node with STATE = problem.INITIAL-STATE.
    first_node = Node(environment.startstate, None, 0, heu)
    
    # Create and initialize the frontier.
    frontier = PriorityQueue()
    frontier.add(first_node)
    
    # An empty set for child already explored.
    explored = set()

    time_cost = 0
    space_cost = 1

    while True:
        # Check if EMPTY?(frontier).
        if frontier.is_empty():
            return None, time_cost, space_cost

        # node <- POP(frontier).
        node = frontier.remove()
        space_cost -= 1
        
        # Check if it is a goal state.
        if node.state == environment.goalstate:
            return build_path(node), time_cost, space_cost

        # Add node.STATE to explored.
        explored.add(node.state)
        space_cost += 1

        # For each action in problem.ACTIONS(environment.ACTION-SPACE) (look around).
        for action in range(environment.action_space.n):
            next_move = environment.sample(node.state, action)
            pos = environment.state_to_pos(next_move)
            Heu.l1_norm(pos, goalpos)

            # Path cost and euristic as node value.
            child = Node(next_move, node, node.pathcost + 1, heu)

            # Check if child.STATE is not in explored.
            if (child.state not in explored):
                # frontier <- INSERT(child, frontier).
                frontier.add(child)
                space_cost += 1

            if frontier.frlen + len(explored) > space_cost:
                space_cost = frontier.frlen + len(explored)
                
            time_cost += 1

The following function calls your implementations of greedy_tree_search and greedy_graph_search:

In [25]:
def greedy(environment, search_type):
    """
    Greedy-best-first search
    
    Args:
        problem: OpenAI Gym environment
        search_type: type of search - greedy_tree_search or greedy_graph_search (function pointer)
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    path, time_cost, space_cost = search_type(environment)
    return path, time_cost, space_cost

The following code calls your tree search and graph search version of Greedy-best-first search and prints the results:

In [26]:
envname = "SmallMaze-v0"
environment = gym.make(envname)

solution_ts, time_ts, memory_ts = greedy(environment, greedy_tree_search)
solution_gs, time_gs, memory_gs = greedy(environment, greedy_graph_search)

print("\n----------------------------------------------------------------")
print("\tGREEDY BEST FIRST TREE SEARCH PROBLEM: ")
print("----------------------------------------------------------------")
print("Solution: {}".format(solution_2_string(solution_ts, environment)))
print("N° of nodes explored: {}".format(time_ts))
print("Max n° of nodes in memory: {}".format(memory_ts))

print("\n----------------------------------------------------------------")
print("\tGREEDY BEST FIRST GRAPH SEARCH PROBLEM: ")
print("----------------------------------------------------------------")
print("Solution: {}".format(solution_2_string(solution_gs, environment)))
print("N° of nodes explored: {}".format(time_gs))
print("Max n° of nodes in memory: {}".format(memory_gs))


----------------------------------------------------------------
	GREEDY BEST FIRST TREE SEARCH PROBLEM: 
----------------------------------------------------------------
Solution: time-out
N° of nodes explored: 10000
Max n° of nodes in memory: 30001

----------------------------------------------------------------
	GREEDY BEST FIRST GRAPH SEARCH PROBLEM: 
----------------------------------------------------------------
Solution: [(0, 1), (1, 1), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 72
Max n° of nodes in memory: 18


Correct results can be found [here](lesson_2_results.txt).

## Assignment 2: A* Search
The second assignment is to implement the A search algorithm on SmallMaze. In particular, you are required to implement both astar_tree_search and astar_graph_search versions that will be called by the generic astar. Use L1 norm* as heuristic function at first, then try also the others to see the differences.

The results returned by astar must be in the following form (path, time_cost, space_cost), more in detail:

- **path** - tuple of state identifiers forming a path from the start state to the goal state. None if no solution is found.
- **time_cost** - the number of nodes checked during the exploration.
- **space_cost** - the maximum number of nodes in memory at the same time.

Functions to implement:
- *astar_tree_search(environment)*
- *astar_graph_search(environment)*

In [8]:
def astar_tree_search(environment):
    """
    A* Tree search

    Args:
        problem: OpenAI Gym environment

    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """

    goalpos = environment.state_to_pos(environment.goalstate)
    start_pos = environment.startstate

    # Mahnattan distance.
    heu = Heu.l1_norm(start_pos, goalpos)
    
    # The node with STATE = problem.INITIAL-STATE.
    first_node = Node(environment.startstate, None, 0, heu)
    
    # Create and initialize the frontier.
    frontier = PriorityQueue()
    frontier.add(first_node)

    time_cost = 0
    space_cost = 1

    while True:
        # Check if EMPTY?(frontier).
        if frontier.is_empty():
            return None, time_cost, space_cost

        # node <- POP(frontier).
        node = frontier.remove()
        time_cost += 1

        # Check if node.STATE is a problem.GOAL.
        if node.state == environment.goalstate:
            return build_path(node), time_cost, space_cost

        # For each action in problem.ACTIONS(environment.ACTION-SPACE) (look around).
        for action in range(environment.action_space.n):
            next_move = environment.sample(node.state, action)
            pos = environment.state_to_pos(next_move)
            heu = Heu.l1_norm(pos, goalpos)

            # h = h* x g
            child = Node(next_move, 
                         node, 
                         node.pathcost + 1, 
                         node.pathcost + heu + 1)
            
            # frontier <- INSERT(child, frontier).
            frontier.add(child)

            if frontier.frlen > space_cost:
                space_cost = frontier.frlen

In [9]:
def astar_graph_search(environment):
    """
    A* Graph search

    Args:
        problem: OpenAI Gym environment

    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """

    goalpos = environment.state_to_pos(environment.goalstate)
    start_pos = environment.startstate

    # Mahnattan distance.
    heu = Heu.l1_norm(start_pos, goalpos)
    
    # The node with STATE = problem.INITIAL-STATE.
    first_node = Node(environment.startstate, None, 0, heu)
    
    # Create and initialize the frontier.
    queue = PriorityQueue()
    queue.add(first_node)
    
    # An empty set for child already explored.
    explored = set()
    
    time_cost = 0
    space_cost = 1

    while True:
        # Check if EMPTY?(frontier).
        if queue.is_empty():
            return None, time_cost, space_cost

        # node <- POP(frontier).
        node = queue.remove()

        # Add node.STATE to explored.
        explored.add(node.state)
        time_cost += 1

        # Check if it is a goal state.
        if node.state == environment.goalstate:
            return build_path(node), time_cost, space_cost

        # For each action in problem.ACTIONS(environment.ACTION-SPACE) (look around).
        for action in range(environment.action_space.n):
            next_move = environment.sample(node.state, action)
            pos = environment.state_to_pos(next_move)
            heu = Heu.l1_norm(pos, goalpos)

            # f = h x g
            child = Node(next_move, 
                         node, 
                         node.pathcost + 1, 
                         node.pathcost + heu + 1)

            # Check if child.STATE is not in explored.
            if child.state not in explored:
                queue.add(child)

                if queue.frlen > space_cost:
                    space_cost = queue.frlen

The following function calls your implementations of astar_tree_search and astar_graph_search:

In [10]:
def astar(environment, search_type):
    """
    A* search
    
    Args:
        environment: OpenAI Gym environment
        search_type: type of search - astar_tree_search or astar_graph_search (function pointer)
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    path, time_cost, space_cost = search_type(environment)
    return path, time_cost, space_cost

The following code calls your tree search and graph search version of A* search and prints the results:

In [11]:
envname = "SmallMaze-v0"
environment = gym.make(envname)

solution_ts, time_ts, memory_ts = astar(environment, astar_tree_search)
solution_gs, time_gs, memory_gs = astar(environment, astar_graph_search)

print("\n----------------------------------------------------------------")
print("\tA* TREE SEARCH PROBLEM: ")
print("----------------------------------------------------------------")
print("Solution: {}".format(solution_2_string(solution_ts, environment)))
print("N° of nodes explored: {}".format(time_ts))
print("Max n° of nodes in memory: {}".format(memory_ts))

print("\n----------------------------------------------------------------")
print("\tA* GRAPH SEARCH PROBLEM: ")
print("----------------------------------------------------------------")
print("Solution: {}".format(solution_2_string(solution_gs, environment)))
print("N° of nodes explored: {}".format(time_gs))
print("Max n° of nodes in memory: {}".format(memory_gs))


----------------------------------------------------------------
	A* TREE SEARCH PROBLEM: 
----------------------------------------------------------------
Solution: [(0, 1), (1, 1), (2, 1), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 2091
Max n° of nodes in memory: 6271

----------------------------------------------------------------
	A* GRAPH SEARCH PROBLEM: 
----------------------------------------------------------------
Solution: [(0, 1), (1, 1), (2, 1), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 22
Max n° of nodes in memory: 5


Correct results can be found [here](lesson_2_results.txt).

### Discussion
Now that you have correctly implemented both Greedy-best-first and A* what can you say about the solutions they compute? Are there significant differences in the stats? Try to play with other heuristics as well and see if your results change.